<a href="https://colab.research.google.com/github/agroimpacts/VegMapper/blob/dev-calval-simplify/calval/process_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a single training/validation/test set from multiple Collect Earth projects



### Table of Contents

* [Overview](#overview)
* [Set-up](#setup)
* [Sample preparation](#sample-prep)
    * [Read-in, reshape, and recode](#reshape-recode)
    * [Simplify the classes](#simplify)
    * [Calculate sample agreement](#agreement)
* [Split the dataset](#split)
    * [Combine and convert to spatial](#combine)

## Overview <a class="anchor" id="overview"></a>
This notebook demonstrates how several Collect Earth Online projects can be:

1. Provide functionality to check the structure and validity of user input for modeling; Yet, users are responsible for providing data in good format.
2. Re-code the class values and rename the column names.
2. Merged into a single dataset that provide a single label for each sample point and an estimate of label uncertainty;
3. Split into training, validation, and test (or map reference) samples.

The data used in this demonstration are the results of three Collect Earth Online projects that were captured over the Department of Ucayali, Peru. Each project represents the efforts of an individual (or group of individuals working in the same project) to label 1350 points, classifying each into 1 of 4 classes: not oil palm; young oil palm; mature oil palm;  unsure. The datasets preserve all the information from these projects, although user email addresses were anonymized.

## Sample preparation <a class="anchor" id="sample-prep"></a>
Load packages, setup configuations, define a helper function...

In [1]:
# from label_utils import load_csv, subset_cols, rename_cols,\
#     check_exclusive, recode, combine_labelers, get_mode_and_occurence

#@title (RUN) Setup code
## Mount Drive
from google.colab import drive
root = '/content/gdrive'
drive.mount(root)

## Clone and/or update VegMapper
import os
# from datetime import datetime as dt
import pandas as pd
from sklearn.model_selection import train_test_split

repo_path = f"{root}/MyDrive/repos"
clone_path = 'https://github.com/agroimpacts/VegMapper.git'
if not os.path.exists(repo_path):
    print(f"Making {repo_path}")
    os.makedirs(repo_path, exist_ok=True)

if not os.path.exists(f"{repo_path}/VegMapper"):
    !git -C "{repo_path}" clone "{clone_path}"
else:
    !git -C "{repo_path}/VegMapper" pull

os.chdir(f"{repo_path}/VegMapper")

# Import sample_utils function
from vegmapper.calval.label_utils import *
from functools import partial

Mounted at /content/gdrive
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 10 (delta 8), reused 6 (delta 4), pack-reused 0
Unpacking objects: 100% (10/10), 1.05 KiB | 0 bytes/s, done.
From https://github.com/agroimpacts/VegMapper
   b3df57e..f4f44f5  dev-calval-simplify -> origin/dev-calval-simplify
Updating c434ee4..f4f44f5
error: Your local changes to the following files would be overwritten by merge:
	vegmapper/calval/label_utils.py
	vegmapper/calval/sample_utils.py
Please commit your changes or stash them before you merge.
Aborting


In [2]:
# @title Loading CSVs

while True:
    try:
        num_users = int(input("Enter the number of CEO projects (The number of CEO projects must be more than 2): "))
        if num_users < 2:
            print("The number of CEO projects must be at least 2.")
        else:
            break
    except ValueError:
        print("Invalid input. Please enter a valid number.")

# Initialize the list to store file paths
fs = []

# Loop to input file paths for each user
for i in range(num_users):
    while True:
        file_path = input(f"Enter the CSV file name for user {i+1}: ")
        if not file_path:
            print("File name cannot be empty.")
        else:
            fs.append(file_path)
            break

Enter the number of CEO projects (The number of CEO projects must be more than 2): /content/gdrive/MyDrive/VegMapper/calval/data/ceo-survey-user1.csv
Invalid input. Please enter a valid number.
Enter the number of CEO projects (The number of CEO projects must be more than 2): 3
Enter the CSV file name for user 1: /content/gdrive/MyDrive/VegMapper/calval/data/ceo-survey-user1.csv
Enter the CSV file name for user 2: /content/gdrive/MyDrive/VegMapper/calval/data/ceo-survey-user2.csv
Enter the CSV file name for user 3: /content/gdrive/MyDrive/VegMapper/calval/data/ceo-survey-user3.csv


In [4]:
# @title Defining columns

df = pd.read_csv(fs[0])

print("\nColumn Names and Indices:")
for i, column in enumerate(df.columns):
    print(f"{i}: {column}")

new_col_names_input = input("Enter the new column names (comma-separated): ")
new_col_names = [name.strip() for name in new_col_names_input.split(',')]
new_col_names

presence_columns_input = input(f"Enter column indices to change to '{new_col_names[0]}' (comma-separated): ")
presence_columns = [index.strip() for index in presence_columns_input.split(',')]

absence_columns_input = input(f"Enter column indices to change to '{new_col_names[1]}' (comma-separated): ")
absence_columns = [index.strip() for index in absence_columns_input.split(',')]

not_sure_columns_input = input(f"Enter column indices to change to '{new_col_names[2]}' (comma-separated): ")
not_sure_columns = [index.strip() for index in not_sure_columns_input.split(',')]


rename_dict = {
    "plot_id": "Point_ID",
    "pl_cluster": "Clust",
    "center_lat": "Lat",
    "center_lon": "Lon"
}
for i, column in enumerate(df.columns):
    if str(i) in presence_columns:
        rename_dict[column] = new_col_names[0]
    elif str(i) in absence_columns:
        rename_dict[column] = new_col_names[1]
    elif str(i) in not_sure_columns:
        rename_dict[column] = new_col_names[2]

print(rename_dict)
print(new_col_names)


Column Names and Indices:
0: plot_id
1: center_lon
2: center_lat
3: size_m
4: shape
5: sample_points
6: email
7: flagged
8: flagged_reason
9: collection_time
10: analysis_duration
11: common_securewatch_date
12: total_securewatch_dates
13: pl_plotid
14: pl_cluster
15: Oil Palm?:Young Oil Palm
16: Oil Palm?:Mature Oil Palm
17: Oil Palm?:Not Oil Palm
18: Oil Palm?:Not Sure
Enter the new column names (comma-separated): Presence, Absence, Not Sure
Enter column indices to change to 'Presence' (comma-separated): 15, 16
Enter column indices to change to 'Absence' (comma-separated): 17
Enter column indices to change to 'Not Sure' (comma-separated): 18
{'plot_id': 'Point_ID', 'pl_cluster': 'Clust', 'center_lat': 'Lat', 'center_lon': 'Lon', 'Oil Palm?:Young Oil Palm': 'Presence', 'Oil Palm?:Mature Oil Palm': 'Presence', 'Oil Palm?:Not Oil Palm': 'Absence', 'Oil Palm?:Not Sure': 'Not Sure'}
['Presence', 'Absence', 'Not Sure']


In [5]:
# @title Defining which columns represents presence (1), absence (0) or unsure (2)

recode_dict = {}

# Iterate through new_col_names and get user input for values
for column_name in new_col_names:
    arbitrary_number = int(input(f"Enter a number for the '{column_name}' category: "))
    recode_dict[column_name] = arbitrary_number

print("Updated recode_dict:")
print(recode_dict)

Enter a number for the 'Presence' category: 1
Enter a number for the 'Absence' category: 0
Enter a number for the 'Not Sure' category: 2
Updated recode_dict:
{'Presence': 1, 'Absence': 0, 'Not Sure': 2}


### Read-in, reshape, and recode classes <a class="anchor" id="reshape-recode"></a>
The first step was to combine the three datasets into a single dataset, with the columns from each of the three CEO projects, and to recode the four classes into a single column with values 0 (absence),  1 (presence), and 2 (unsure). At this step, we end up with 3 columns, 1 per completed CEO project: `cl1` = samples from project 1, `cl2` = samples from project 2, `cl3` = samples from project 3. Each column contains the recoded classes (note the renaming of the columns is done in the next code chunk).

### Simplify the classes <a class="anchor" id="simplify"></a>

In this step, a single classification is created by finding the modal class for each sample point across the 3 groups' results. This creates a new `class` column, which provides the class from the majority opinion.

We repeat this same step again after first collapsing, within each of the `cl1:cl3` columns, the two oil palm classes into a single *oil palm* class with value = 1--*not oil palm* remains 0, and *unsure* remains 3. The modal function was re-run to create a new consensus class, called `class2`. We recommend that `class2` be used for modelling, while `class` may be useful for understanding error patterns.

In [6]:
# @title Process and combine csvs
label_name = "labeler"

# Define a partial function with fixed arguments
process_csv_partial = partial(process_csv, rename_dict=rename_dict,
                              recode_dict=recode_dict, new_col_names=new_col_names)

# Process ceo-survey-users one by one
dats = list(map(process_csv_partial, fs))

# Combine three datasets into one
combined = combine_labelers(dats, by=["Point_ID", "Clust"], label_name=label_name, fs=fs)
combined.head()

processing: /content/gdrive/MyDrive/VegMapper/calval/data/ceo-survey-user1.csv
The labeled classes are mutually exclusive.
processing: /content/gdrive/MyDrive/VegMapper/calval/data/ceo-survey-user2.csv
The labeled classes are mutually exclusive.
processing: /content/gdrive/MyDrive/VegMapper/calval/data/ceo-survey-user3.csv


/content/gdrive/MyDrive/repos/VegMapper/vegmapper/calval/label_utils.py:110: UserWarning: Found at least one entry(s) that does not have mutually exclusive labels.
        >>>file: /content/gdrive/MyDrive/VegMapper/calval/data/ceo-survey-user3.csv<<<
        Check your columns values.
        (1)Make sure no empty entry in those columns.
        (2)Make sure there is one and only one column is labeled as 100.
  warnings.warn('Found at least one entry(s) that does not have mutually exclusive labels.\n\


,Point_ID,Clust,Lat,Lon,ceo-survey-user1,ceo-survey-user2,ceo-survey-user3
0,140884433,1,-8.321900,-75.045545,1,1,1
1,140884434,1,-8.344409,-74.884792,0,0,0
2,140884435,0,-8.836094,-74.342566,0,0,0
3,140884436,1,-8.321630,-75.031377,1,1,1
4,140884437,0,-10.948943,-71.736808,0,0,0


### Calculate sample agreement <a class="anchor" id="agreement"></a>
The next step was to calculate some agreement metrics across the three groups' samples. The primary approach was to calculate the proportion of labelling teams that selected the modal class. Since there were just three teams in this example, values were either 0.333, 0.667, 1. This agreement was calculated across for both the original classification scheme (class: 0-3) and the simplifed scheme (), with columns `agree` and `agree2` providing the respective proportions for each observation.

In [14]:
# @title Agreement mode

num_labelers = len(fs)
labels = []

for i in range(1, num_labelers + 1):
    file_name = os.path.splitext(os.path.basename(fs[i-1]))[0]
    labels.append(file_name)

combined[['mode', 'mode_agreement']] = combined[labels].apply(get_mode_and_occurence, axis=1, result_type='expand')
pd.set_option('display.max_rows', None)
#print(combined)

# we can set the mode to -9999 if there is no agreement (mode_freq = 1/num_labelers)
combined.loc[combined['mode_agreement'] <= 1/3, 'mode'] = -9999

combined = combined.drop(combined[combined['mode'] == -9999].index)
print(combined.shape)

combined.head()

(1342, 9)


,Point_ID,Clust,Lat,Lon,ceo-survey-user1,ceo-survey-user2,ceo-survey-user3,mode,mode_agreement
0,140884433,1,-8.321900,-75.045545,1,1,1,1.0,1.0
1,140884434,1,-8.344409,-74.884792,0,0,0,0.0,1.0
2,140884435,0,-8.836094,-74.342566,0,0,0,0.0,1.0
3,140884436,1,-8.321630,-75.031377,1,1,1,1.0,1.0
4,140884437,0,-10.948943,-71.736808,0,0,0,0.0,1.0


We can then calculate the average agreement per sample to get a sense of the uncertainty in labels for each class, for all 4 classes

In [15]:
# @title Average agreement per sample
agreement = combined.groupby("mode").mean()
agreement = agreement.rename(columns={"mode_agreement": "mean agreement"})
print(agreement[['mean agreement']])

      mean agreement
mode                
0.0         0.988607
1.0         0.894253
2.0         0.690476


In [16]:
# @title Check the result
combined.head()

,Point_ID,Clust,Lat,Lon,ceo-survey-user1,ceo-survey-user2,ceo-survey-user3,mode,mode_agreement
0,140884433,1,-8.321900,-75.045545,1,1,1,1.0,1.0
1,140884434,1,-8.344409,-74.884792,0,0,0,0.0,1.0
2,140884435,0,-8.836094,-74.342566,0,0,0,0.0,1.0
3,140884436,1,-8.321630,-75.031377,1,1,1,1.0,1.0
4,140884437,0,-10.948943,-71.736808,0,0,0,0.0,1.0


And for the reduced set of classes

## Split the dataset <a class="anchor" id="split"></a>

Here we split the dataset into three parts for model training (60% of the sample), validation (20%), and final assessment (the 20% set aside as the test or map reference dataset).

The splits are confined to the usable sample, which is defined as samples not falling into class 3 and those with at least 2/3 observers agreeing on the class. This decision is made based on the simplified sample scheme (class2), rather than the full scheme (class). The resulting splits are denoted in a column called `usage` (this is distinct from the column `use`, which was used to filter out unusable observations).

Values of "unusable" in the `usage` column indicate observations that were not usable because of their low agreement or uncertain class.  They are included here for completeness, and in case they help with evaluation


In [17]:
# @title Run the split

#Set random seed for train/validation/reference split
seed = 999
train, test = train_test_split(combined, test_size=0.4, train_size=0.6,
                               random_state=seed)

val, ref = train_test_split(test, test_size=0.5, train_size=0.5,
                            random_state=seed)

out = pd.concat(
    [train.assign(usage="train"), val.assign(usage="validate"),
     ref.assign(usage="map_reference/test")]
).reset_index(drop=True)
out.head()

,Point_ID,Clust,Lat,Lon,ceo-survey-user1,ceo-survey-user2,ceo-survey-user3,mode,mode_agreement,usage
0,140884459,0,-10.661994,-71.784007,0,0,0,0.0,1.000000,train
1,140884500,0,-9.320213,-73.921025,0,0,0,0.0,1.000000,train
2,140884488,1,-8.493207,-74.608635,0,0,0,0.0,1.000000,train
3,140885706,1,-8.263247,-74.816985,0,0,0,0.0,1.000000,train
4,140884597,1,-8.517548,-74.931080,2,2,1,2.0,0.666667,train


### Combine and export to csv <a class="anchor" id="combine"></a>

The ineligible portion of the sample is also added back for completeness

In [ ]:
#@title (RUN) Export sample

gdrive_folder = input(f"Enter the name of the output folder: \n\n")
csv_name = input(f"Enter the name of the output csv file: \n\n")

output_dir = f"{root}/MyDrive/{gdrive_folder}"
os.makedirs(output_dir, exist_ok=True)

outpath = os.path.join(output_dir, csv_name)

with open(outpath, 'w') as f:
    out.to_csv(f, float_format='{:f}'.format, encoding='utf-8', index=False)

print('file exported')

And their locations on a map

In [98]:
# @title Display results

# For the usage category
color_mapping = {
    'train': 'blue',
    'validate': 'green',
    'map_reference/test': 'red',
    'unusable': 'gray'
}

m = folium.Map(location=[out['Lat'].mean(), out['Lon'].mean()], zoom_start=7)
scatter_group_usage = folium.FeatureGroup(name='Usage')

scatter_group_mode = folium.FeatureGroup(name='Mode')

# Create legend for 'usage'
legend_html_usage = '''
<div style="position: fixed; bottom: 50px; left: 50px; background-color: white; border: 2px solid grey; z-index: 9999; padding: 10px;">
    <h4>Usage</h4>
    <i style="background: blue; border-radius: 50%; width: 18px; height: 18px; display: inline-block;"></i> Train<br>
    <i style="background: green; border-radius: 50%; width: 18px; height: 18px; display: inline-block;"></i> Validate<br>
    <i style="background: red; border-radius: 50%; width: 18px; height: 18px; display: inline-block;"></i> Map Reference/Test<br>
    <i style="background: gray; border-radius: 50%; width: 18px; height: 18px; display: inline-block;"></i> Unusable<br>
</div>
'''
legend_usage = folium.Element(legend_html_usage)
m.get_root().html.add_child(legend_usage)

# Create legend for 'mode'
legend_html_mode = '''
<div style="position: fixed; bottom: 50px; left: 230px; background-color: white; border: 2px solid grey; z-index: 9999; padding: 10px;">
    <h4>Mode</h4>
    <i style="background: purple; border-radius: 50%; width: 18px; height: 18px; display: inline-block;"></i> Mode 0<br>
    <i style="background: yellow; border-radius: 50%; width: 18px; height: 18px; display: inline-block;"></i> Mode 1<br>
    <i style="background: black; border-radius: 50%; width: 18px; height: 18px; display: inline-block;"></i> Mode 2<br>
</div>
'''
legend_mode = folium.Element(legend_html_mode)
m.get_root().html.add_child(legend_mode)

for usage, color in color_mapping.items():
    subset = out[out['usage'] == usage]
    for _, row in subset.iterrows():
        folium.CircleMarker(location=[row['Lat'], row['Lon']], radius=1, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(scatter_group_usage)

for mode in range(3):  # 0, 1, 2
    subset = out[out['mode'] == mode]
    color = ['purple', 'yellow', 'black'][mode]
    for _, row in subset.iterrows():
        folium.CircleMarker(location=[row['Lat'], row['Lon']], radius=1, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(scatter_group_mode)

# Add the feature groups to the map
scatter_group_usage.add_to(m)
scatter_group_mode.add_to(m)

# Add OpenStreetMap layer to the map
folium.TileLayer('openstreetmap').add_to(m)

# Add a layer control to toggle between 'usage' and 'mode' scatter plots and OpenStreetMap layer
folium.LayerControl(collapsed=False).add_to(m)

m